In [1]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [3]:
# Model structure
class Net(nn.Module):
    def __init__(self):
        # 在构造函数中，实例化不同的layer组件，并赋给类成员变量
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 在前馈函数中，利用实例化的组件对网络进行搭建，并对输入Tensor进行操作，并返回Tensor类型的输出结果
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        return x


In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('Running Device:', device)

Running Device: cpu


In [13]:
net = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
# 对获取的图像数据做ToTensor()变换和归一化
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 利用torchvision提供的CIFAR10数据集类，实例化训练集和测试集提取类
trainset = torchvision.datasets.CIFAR10(root='./data1', train=True, download=False, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data1', train=False, download=False, transform=transform)

# 利用torch提供的DataLoader, 实例化训练集DataLoader 和 测试集DataLoader
Batch_size = 2
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=Batch_size, shuffle=True, num_workers=2)
testLoader = torch.utils.data.DataLoader(testset, batch_size=Batch_size, shuffle=False, num_workers=2)

# CIFAR10 类别内容
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print("OK")

OK


In [21]:
#测试 trainLoader，读取一个batch的数据
data_iter = iter(trainLoader)
inputs, labels = next(data_iter)

print("inputs.shape = ", inputs.shape)
print("labels.shape = ", labels.shape)
print("inputs = ", inputs)
print("labels = ", labels)


inputs.shape =  torch.Size([2, 3, 32, 32])
labels.shape =  torch.Size([2])
inputs =  tensor([[[[-0.4667, -0.5529, -0.5765,  ..., -0.4588, -0.4667, -0.4980],
          [-0.4510, -0.3490, -0.4902,  ..., -0.4824, -0.4667, -0.4824],
          [-0.4039,  0.3490,  0.4902,  ..., -0.4588, -0.4431, -0.4667],
          ...,
          [ 0.0275,  0.0745,  0.0980,  ...,  0.2235,  0.1216,  0.1216],
          [-0.0745, -0.0588,  0.1137,  ...,  0.2314,  0.2471,  0.2000],
          [ 0.2157,  0.3020,  0.4196,  ...,  0.2706,  0.3333,  0.2706]],

         [[-0.2627, -0.3255, -0.3804,  ..., -0.2471, -0.2549, -0.2863],
          [-0.3333, -0.3098, -0.4118,  ..., -0.2784, -0.2549, -0.2706],
          [-0.3804, -0.1216, -0.1294,  ..., -0.2471, -0.2314, -0.2549],
          ...,
          [-0.1216, -0.0824, -0.0510,  ...,  0.1686,  0.1059,  0.1216],
          [-0.1843, -0.1843, -0.0353,  ...,  0.1294,  0.1608,  0.1216],
          [ 0.0353,  0.1137,  0.2157,  ...,  0.1686,  0.2314,  0.1843]],

         [[-0.302

In [22]:
#一个批次做前向，并且预测标签
outputs = net(inputs)
print(outputs)

tensor([[ 0.1144, -0.0021, -0.0333,  0.0388, -0.0031,  0.0453,  0.0273, -0.1251,
         -0.0931,  0.0633],
        [ 0.1268,  0.0034, -0.0369,  0.0439, -0.0198,  0.0368,  0.0332, -0.1233,
         -0.1113,  0.0511]], grad_fn=<AddmmBackward0>)


In [18]:
#一个批次做前向，并且预测标签；如果是测试阶段，则不需要保留导数、创建计算图
with torch.no_grad():
    outputs = net(inputs)
    print(outputs)

tensor([[-0.0431, -0.0450, -0.0058, -0.0662,  0.0256,  0.0638, -0.0228,  0.0704,
         -0.0085,  0.1118],
        [-0.0492, -0.0564,  0.0011, -0.0664,  0.0376,  0.0560, -0.0221,  0.0656,
         -0.0062,  0.1068]])


In [ ]:
#获得一个批次的预测
#分数最大的作为预测结果
_, predicted = torch.max(outputs.data, 1)
print(predicted)

predicted = torch.argmax(outputs.data, 1)
print(predicted)

tensor([9, 9])
tensor([9, 9])


In [22]:
#计算一个批次的CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

print(outputs)
print(labels)

loss = criterion(outputs, labels)

print(loss.item())

tensor([[-0.0431, -0.0450, -0.0058, -0.0662,  0.0256,  0.0638, -0.0228,  0.0704,
         -0.0085,  0.1118],
        [-0.0492, -0.0564,  0.0011, -0.0664,  0.0376,  0.0560, -0.0221,  0.0656,
         -0.0062,  0.1068]])
tensor([9, 7])
2.222756862640381


In [26]:
#计算准确率

print(predicted == labels)
correct = (predicted == labels).sum().item()
acc = correct / labels.size()[0]
print(acc)



tensor([ True, False])
0.5


In [27]:
#遍历 trainLoader，逐一读取
for i, data in enumerate(trainLoader, 0):
    if i > 2:
        break 
    inputs, labels = data
    print("i = ", i )
    print("inputs.shape = ", inputs.shape)
    print("labels.shape = ", labels.shape)
    print("inputs = ", inputs)
    print("labels = ", labels)
    
    # 将数据迁移到device中，如device为GPU，则将数据从CPU迁移到GPU；如device为CPU，则将数据从CPU迁移到CPU（即不作移动）
    inputs, labels = inputs.to(device), labels.to(device)

i =  0
inputs.shape =  torch.Size([2, 3, 32, 32])
labels.shape =  torch.Size([2])
inputs =  tensor([[[[ 1.0000,  1.0000,  1.0000,  ...,  0.5686,  0.6235,  0.9137],
          [ 1.0000,  0.9765,  0.9922,  ...,  0.6863,  0.6078,  0.5686],
          [ 1.0000,  1.0000,  0.9216,  ...,  0.4980,  0.6078,  0.4667],
          ...,
          [ 1.0000,  0.9922,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  0.9922,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  0.9922,  1.0000,  ...,  1.0000,  1.0000,  1.0000]],

         [[ 1.0000,  1.0000,  1.0000,  ...,  0.0275,  0.2941,  0.8039],
          [ 1.0000,  0.9765,  0.9922,  ..., -0.0588, -0.1137,  0.1765],
          [ 1.0000,  1.0000,  0.9216,  ..., -0.1608, -0.1529, -0.1843],
          ...,
          [ 1.0000,  0.9922,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  0.9922,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  0.9922,  1.0000,  ...,  1.0000,  1.0000,  1.0000]],

         [

In [14]:
#遍历 testLoader，逐一读取
for i, data in enumerate(testLoader, 0):
    if i > 0:
        break 
    inputs, labels = data
    print("inputs.shape = ", inputs.shape)
    print("labels.shape = ", labels.shape)
    print("inputs = ", inputs)
    print("labels = ", labels)
    
    # 将数据迁移到device中，如device为GPU，则将数据从CPU迁移到GPU；如device为CPU，则将数据从CPU迁移到CPU（即不作移动）
    inputs, labels = inputs.to(device), labels.to(device)

inputs.shape =  torch.Size([2, 3, 32, 32])
labels.shape =  torch.Size([2])
inputs =  tensor([[[[ 0.2392,  0.2471,  0.2941,  ...,  0.0745, -0.0118, -0.0902],
          [ 0.1922,  0.1843,  0.2471,  ...,  0.0667, -0.0196, -0.0667],
          [ 0.1843,  0.1843,  0.2392,  ...,  0.0902,  0.0196, -0.0588],
          ...,
          [-0.4667, -0.6706, -0.7569,  ..., -0.7020, -0.8980, -0.6863],
          [-0.5216, -0.6157, -0.7255,  ..., -0.7961, -0.7725, -0.8431],
          [-0.5765, -0.5608, -0.6471,  ..., -0.8118, -0.7333, -0.8353]],

         [[-0.1216, -0.1294, -0.0902,  ..., -0.2549, -0.2863, -0.3333],
          [-0.1216, -0.1373, -0.1059,  ..., -0.2549, -0.2863, -0.3098],
          [-0.1373, -0.1451, -0.1294,  ..., -0.2314, -0.2549, -0.3020],
          ...,
          [-0.0275, -0.2157, -0.3098,  ..., -0.2392, -0.4980, -0.3333],
          [-0.0902, -0.2000, -0.3333,  ..., -0.3569, -0.3569, -0.4980],
          [-0.1608, -0.1765, -0.3020,  ..., -0.3961, -0.3412, -0.4745]],

         [[-0.615